# Chapter 10: Sparse matrices and graphs

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['text.usetex'] = True

In [3]:
import scipy.sparse as sp
import scipy.sparse.linalg
import scipy.linalg as la
import numpy as np

In [4]:
import networkx as nx

![pic](pics/sparse-matrix-representations.png)

### Coordinate list format
* store lists of column & row indices together with a list of nonzero values.

In [5]:
values = [1, 2, 3, 4]
rows   = [0, 1, 2, 3]
cols   = [1, 3, 2, 0]

In [6]:
# coo_matrix() == coordinate list format
A = sp.coo_matrix((values, (rows, cols)), shape=[4, 4])
A.todense()

matrix([[0, 1, 0, 0],
        [0, 0, 0, 2],
        [0, 0, 3, 0],
        [4, 0, 0, 0]])

In [7]:
# various attributes
A.shape, A.size, A.dtype, A.ndim

((4, 4), 4, dtype('int64'), 2)

In [8]:
A.nnz, A.data, A.row, A.col

(4,
 array([1, 2, 3, 4]),
 array([0, 1, 2, 3], dtype=int32),
 array([1, 3, 2, 0], dtype=int32))

In [9]:
# convert to CSR (compressed sparse row) format)
A.tocsr()

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [10]:
A.toarray()

array([[0, 1, 0, 0],
       [0, 0, 0, 2],
       [0, 0, 3, 0],
       [4, 0, 0, 0]])

In [11]:
A.todense()

matrix([[0, 1, 0, 0],
        [0, 0, 0, 2],
        [0, 0, 3, 0],
        [4, 0, 0, 0]])

Not all sparse matrix formats supports indexing:

In [12]:
# Not all sparse matrix formats support indexing
#A[1,2]

In [13]:
# Not all sparse matrix formats support indexing
#A.tobsr()[1,2]

But some do:

In [14]:
A.tocsr()[1,2]

0

In [15]:
A.tolil()[1:3,3]

<2x1 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in LInked List format>

### CSR (Compressed Sparse Row format)
* Along with CSC (compressed sparse column), best suited for computation tasks

In [16]:
A = np.array([[1, 2, 0, 0], [0, 3, 4, 0], [0, 0, 5, 6], [7, 0, 8, 9]]); A

array([[1, 2, 0, 0],
       [0, 3, 4, 0],
       [0, 0, 5, 6],
       [7, 0, 8, 9]])

In [17]:
A = sp.csr_matrix(A)
A.data

array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [18]:
# column indices coresponding to A's nonzero values
A.indices

array([0, 1, 1, 2, 2, 3, 0, 2, 3], dtype=int32)

In [19]:
# starting positions, per row, of column indices in A
# plus total number of nonzero elements added to end of array
A.indptr

array([0, 2, 4, 6, 9], dtype=int32)

In [20]:
i = 2

In [21]:
A.indptr[i], A.indptr[i+1]-1

(4, 5)

In [22]:
A.indices[A.indptr[i]:A.indptr[i+1]]

array([2, 3], dtype=int32)

In [23]:
A.data[A.indptr[i]:A.indptr[i+1]]

array([5, 6], dtype=int64)

### Functions for constructing sparse matrices
* Often more convenient to use templates in sp.sparse
* sp.eye (ones on diagonal)
* sp.diags (spec'd pattern on diagonal)
* sp.kron  (kronecker tensor product of 2 sparse matrices)
* sp.bmat, sp.vstack, sp.hstack

In [24]:
N = 10

In [25]:
A = -2 * sp.eye(N) + sp.eye(N, k=1) + sp.eye(N, k=-1)

In [26]:
A.todense()

matrix([[-2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.]])

In [27]:
#fig, ax = plt.subplots()
#ax.spy(A)
#fig.savefig("ch10-sparse-matrix-1.pdf");

In [28]:
A = sp.diags([1,-2,1], [1,0,-1], shape=[N, N], format='csc')

In [29]:
A.todense()

matrix([[-2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -2.]])

In [30]:
#fig, ax = plt.subplots()
#ax.spy(A);

In [31]:
B = sp.diags([1, 1], [-1, 1], shape=[3,3])
B.todense()

matrix([[ 0.,  1.,  0.],
        [ 1.,  0.,  1.],
        [ 0.,  1.,  0.]])

In [32]:
B

<3x3 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements (2 diagonals) in DIAgonal format>

In [33]:
C = sp.kron(A, B, format='csr')
C

<30x30 sparse matrix of type '<class 'numpy.float64'>'
	with 112 stored elements in Compressed Sparse Row format>

In [34]:
#fig, (ax_A, ax_B, ax_C) = plt.subplots(1, 3, figsize=(12, 4))
#ax_A.spy(A)
#ax_B.spy(B)
#ax_C.spy(C)
#fig.savefig("ch10-sparse-matrix-2.pdf");

![pic](pics/sparse-a-b-tensorprod.png)

### Sparse linear algebra
* use case: large matrices == not well suited to dense matrix designs
* scipy.sparse.linalg is NOT equal to scipy.linalg (example: dense eigenvalue solvers typically return ALL results; sparse eigenvalue solvers return a subset, typically smallest or largest results.)

In [35]:
# typical use case: solving Ax=b (A = sparse matrix, x&b = dense vectors)
# linalg has direct & iterative solvers (sp.linalg.spsolve)
# linalg also has matrix LU factorization methods (sp.linalg.splu, sp.linalg.spilu)
N = 10

In [36]:
A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc')
A

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 28 stored elements in Compressed Sparse Column format>

In [37]:
b = -np.ones(N)
b

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [38]:
# direct solver, returns dense Numpy array:
x = sp.linalg.spsolve(A, b)
x

array([  5.,   9.,  12.,  14.,  15.,  15.,  14.,  12.,   9.,   5.])

In [39]:
# to use dense solver, need to convert A to dense array first.
np.linalg.solve(A.todense(), b)

array([  5.,   9.,  12.,  14.,  15.,  15.,  14.,  12.,   9.,   5.])

* N=100 is typical performance crossover point
![pic](pics/dense-vs-sparse.png)

In [40]:
# LU factorization; returns L & U factors; has method to solve LUx=b for given b.
# use case: solve Ax=b for multiple vectors b
lu = sp.linalg.splu(A)

In [41]:
lu.L, lu.U

(<10x10 sparse matrix of type '<class 'numpy.float64'>'
 	with 20 stored elements in Compressed Sparse Column format>,
 <10x10 sparse matrix of type '<class 'numpy.float64'>'
 	with 20 stored elements in Compressed Sparse Column format>)

In [42]:
# now can solve LUx=b
x = lu.solve(b)
x

array([  5.,   9.,  12.,  14.,  15.,  15.,  14.,  12.,   9.,   5.])

In [43]:
lu.perm_r, lu.perm_c

(array([0, 1, 2, 3, 4, 5, 6, 8, 7, 9], dtype=int32),
 array([0, 1, 2, 3, 4, 5, 6, 8, 7, 9], dtype=int32))

In [44]:
#
def sp_permute(A, perm_r, perm_c):
    """ permute rows and columns of A """
    M, N = A.shape
    # row permumation matrix
    Pr = sp.coo_matrix(
        (np.ones(M), (perm_r, np.arange(N)))).tocsr()
    # column permutation matrix
    Pc = sp.coo_matrix((np.ones(M), (np.arange(M), perm_c))).tocsr()
    return Pr.T * A * Pc.T

In [45]:
lu.L * lu.U - A

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Column format>

In [46]:
sp_permute(lu.L * lu.U, lu.perm_r, lu.perm_c) - A

<10x10 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Column format>

In [47]:
#fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 4))
#ax1.spy(lu.L)
#ax2.spy(lu.U)
#ax3.spy(A)

In [48]:
# spsolve() = interface to direct solvers; internally performs matrix factorization

# use_umfpack=True is only effective if scikit-umfpack is installed
# (in which case UMFPACK is the default solver)
x = sp.linalg.spsolve(A, b, use_umfpack=True)
x

array([  5.,   9.,  12.,  14.,  15.,  15.,  14.,  12.,   9.,   5.])

In [49]:
# cg() = conjugate gradient method (iterative optimizer)
x, info = sp.linalg.cg(A, b)
x

array([  5.,   9.,  12.,  14.,  15.,  15.,  14.,  12.,   9.,   5.])

In [50]:
# bicgstab() = biconjugate gradient stabilized method (iterative optimizer)
x, info = sp.linalg.bicgstab(A, b)
x

array([  5.,   9.,  12.,  14.,  15.,  15.,  14.,  12.,   9.,   5.])

In [51]:
# lgsmres() = loose generalized minimum residual method (iterative optimizer)
x, info = sp.linalg.lgmres(A, b)
x

array([  5.,   9.,  12.,  14.,  15.,  15.,  14.,  12.,   9.,   5.])

### Eigenvalue problems
* sparse eigenvalues ==> sp.linalg.eigs()
* singular-value problems ==> sp.linalg.svds()
* Real symmetric or complex hermitian matrices ==> sp.linalg.eigsh()
* return given number of eigenvalues & vectors (not all. default = 6)

In [52]:
# example: find four lowest eigenvalues for sparse matrix of 1D Poisson problem

A = sp.diags(
    [1, -2, 1], 
    [1, 0, -1], 
    shape=[10,10], 
    format='csc')

evals, evecs = sp.linalg.eigs(A, k=4, which='LM')
evals

# evals = array of eigenvalues
# evecs = array (Nxk); columns = eigenvectors corresponding to eigenvalues in evals

array([-3.91898595+0.j, -3.68250707+0.j, -3.30972147+0.j, -2.83083003+0.j])

In [53]:
# (dot product A & a column in evecs) == (same evecs column scaled by corr. eigenvalue)
np.allclose(A.dot(evecs[:,0]), evals[0] * evecs[:,0])

True

### An example of a matrix reording method: Reverse Cuthil McKee

In [54]:
perm = sp.csgraph.reverse_cuthill_mckee(A)
perm

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], dtype=int32)

In [55]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
#ax1.spy(A)
#ax2.spy(sp_permute(A, perm, perm))

### Performance comparison sparse/dense

In [56]:
# compare performance of solving Ax=b vs system size N,
# where A is the sparse matrix for the 1d poisson problem
import time

def setup(N):
    A = sp.diags([1,-2,1], [1,0,-1], shape=[N, N], format='csr')
    b = -np.ones(N)
    return A, A.todense(), b

reps = 10
N_vec = np.arange(2, 300, 1)
t_sparse = np.empty(len(N_vec))
t_dense = np.empty(len(N_vec))
for idx, N in enumerate(N_vec):
    A, A_dense, b = setup(N)
    t = time.time()
    for r in range(reps):
        x = np.linalg.solve(A_dense, b)
    t_dense[idx] = (time.time() - t)/reps
    t = time.time()
    for r in range(reps):
        #x = la.solve(A_dense, b)
        x = sp.linalg.spsolve(A, b, use_umfpack=True)
    t_sparse[idx] = (time.time() - t)/reps
    
#fig, ax = plt.subplots(figsize=(8, 4))
#ax.plot(N_vec, t_dense * 1e3, '.-', label="dense")
#ax.plot(N_vec, t_sparse * 1e3, '.-', label="sparse")
#ax.set_xlabel(r"$N$", fontsize=16)
#ax.set_ylabel("elapsed time (ms)", fontsize=16)
#ax.legend(loc=0)
#fig.tight_layout()
#fig.savefig("ch10-sparse-vs-dense.pdf")

### Eigenvalue problems

In [57]:
N = 10

In [58]:
A = sp.diags([1, -2, 1], [1, 0, -1], shape=[N, N], format='csc')

In [59]:
evals, evecs = sp.linalg.eigs(A, k=4, which='LM')

In [60]:
evals

array([-3.91898595+0.j, -2.83083003+0.j, -3.68250707+0.j, -3.30972147+0.j])

In [61]:
np.allclose(A.dot(evecs[:,0]), evals[0] * evecs[:,0])

True

In [62]:
evals, evecs = sp.linalg.eigsh(A, k=4, which='LM')

In [63]:
evals

array([-3.91898595, -3.68250707, -3.30972147, -2.83083003])

In [64]:
evals, evecs = sp.linalg.eigs(A, k=4, which='SR')

In [65]:
evals

array([-3.91898595+0.j, -3.68250707+0.j, -3.30972147+0.j, -2.83083003+0.j])

In [66]:
np.real(evals).argsort()

array([0, 1, 2, 3])

In [67]:
def sp_eigs_sorted(A, k=6, which='SR'):
    """ compute and return eigenvalues sorted by real value """
    evals, evecs = sp.linalg.eigs(A, k=k, which=which)
    idx = np.real(evals).argsort()
    return evals[idx], evecs[idx]

In [68]:
evals, evecs = sp_eigs_sorted(A, k=4, which='SM')

In [69]:
evals

array([-1.16916997+0.j, -0.69027853+0.j, -0.31749293+0.j, -0.08101405+0.j])

#### Random matrix example

In [70]:
N = 100

In [71]:
x_vec = np.linspace(0, 1, 50)

In [72]:
# seed sp.rand with random_state to obtain a reproducible result
M1 = sp.rand(N, N, density=0.2, random_state=112312321)
M2 = sp.rand(N, N, density=0.2, random_state=984592134)

In [73]:
evals = np.array([sp_eigs_sorted((1-x)*M1 + x*M2, k=25)[0] for x in x_vec])

In [75]:
#fig, ax = plt.subplots(figsize=(8, 4))

#for idx in range(evals.shape[1]):
#    ax.plot(x_vec, np.real(evals[:,idx]), lw=0.5)

#ax.set_xlabel(r"$x$", fontsize=16)
#ax.set_ylabel(r"eig.vals. of $(1-x)M_1+xM_2$", fontsize=16)

#fig.tight_layout()
#fig.savefig("ch10-sparse-eigs.pdf")

### Graphs & Networks
* sparse matrix use case: graphs as adjacency matrices
* Scipy.sparse.csgraph module available
* NetworkX python library == more comprehensive

![pic](pics/networkx-ops.png)

In [76]:
g = nx.MultiGraph()
g.add_node(1)
g.nodes()

[1]

In [77]:
g.add_nodes_from([3, 4, 5])
g.nodes()

[1, 3, 4, 5]

In [78]:
g.add_edge(1, 2)
g.edges()

[(1, 2)]

In [79]:
g.add_edges_from([(3, 4), (5, 6)])
g.edges()

[(1, 2), (3, 4), (5, 6)]

In [80]:
g.add_weighted_edges_from([(1, 3, 1.5), (3, 5, 2.5)])
g.edges(data=True)

[(1, 2, {}),
 (1, 3, {'weight': 1.5}),
 (3, 4, {}),
 (3, 5, {'weight': 2.5}),
 (5, 6, {})]

In [81]:
g.add_weighted_edges_from([(6, 7, 1.5)])
g.nodes(), g.edges()

([1, 2, 3, 4, 5, 6, 7], [(1, 2), (1, 3), (3, 4), (3, 5), (5, 6), (6, 7)])

In [82]:
# let's build graph from Tokyo Metro dataset (JSON)
import numpy as np
import json

In [83]:
with open("tokyo-metro.json") as f:
    data = json.load(f)

# data contents: 
# travel times between stations (travel_times)
# possible xfer points to other lines (transfer)
# line color

In [84]:
data.keys()

dict_keys(['G', 'Y', 'F', 'C', 'N', 'Z', 'H', 'M', 'T'])

In [85]:
data["C"]

{'color': '#149848',
 'transfers': [['C3', 'F15'],
  ['C4', 'Z2'],
  ['C4', 'G2'],
  ['C7', 'M14'],
  ['C7', 'N6'],
  ['C7', 'G6'],
  ['C8', 'M15'],
  ['C8', 'H6'],
  ['C9', 'H7'],
  ['C9', 'Y18'],
  ['C11', 'T9'],
  ['C11', 'M18'],
  ['C11', 'Z8'],
  ['C12', 'M19'],
  ['C18', 'H21']],
 'travel_times': [['C1', 'C2', 2],
  ['C2', 'C3', 2],
  ['C3', 'C4', 1],
  ['C4', 'C5', 2],
  ['C5', 'C6', 2],
  ['C6', 'C7', 2],
  ['C7', 'C8', 1],
  ['C8', 'C9', 3],
  ['C9', 'C10', 1],
  ['C10', 'C11', 2],
  ['C11', 'C12', 2],
  ['C12', 'C13', 2],
  ['C13', 'C14', 2],
  ['C14', 'C15', 2],
  ['C15', 'C16', 2],
  ['C16', 'C17', 3],
  ['C17', 'C18', 3],
  ['C18', 'C19', 3]]}

In [86]:
#data

In [87]:
g = nx.Graph()

for line in data.values():
    g.add_weighted_edges_from(line["travel_times"])
    g.add_edges_from(line["transfers"])

In [88]:
# line xfer edges don't have weights.
# mark them by adding boolean attribute "transfer" to each edge

for n1, n2 in g.edges_iter():
    g[n1][n2]["transfer"] = "weight" not in g[n1][n2]

In [89]:
g.number_of_nodes()

184

In [90]:
g.nodes()[:5]

['G10', 'M6', 'F16', 'M12', 'T16']

In [91]:
g.number_of_edges()

243

In [92]:
g.edges()[:5]

[('G10', 'G9'), ('G10', 'G11'), ('M6', 'm5'), ('M6', 'M7'), ('M6', 'M5')]

In [93]:
# for plotting:
# create lists of edges with transfer edges & on-train edges
# also create list of node colors

on_foot = [edge for edge in g.edges_iter() 
           if g.get_edge_data(*edge)["transfer"]]

on_train = [edge for edge in g.edges_iter() 
            if not g.get_edge_data(*edge)["transfer"]]

colors = [data[n[0].upper()]["color"] 
          for n in g.nodes()]

In [94]:
#fig, ax = plt.subplots(1, 1, figsize=(14, 10))

#pos = nx.graphviz_layout(g, prog="neato")
#nx.draw(g, pos, ax=ax, node_size=200, node_color=colors)
#nx.draw_networkx_labels(g, pos=pos, ax=ax, font_size=6)
#nx.draw_networkx_edges(g, pos=pos, ax=ax, edgelist=on_train, width=2)
#nx.draw_networkx_edges(g, pos=pos, ax=ax, edgelist=on_foot, edge_color="blue")

# removing the default axis on all sides:
#for side in ['bottom','right','top','left']:
#    ax.spines[side].set_visible(False)

# removing the axis labels and ticks
#ax.set_xticks([])
#ax.set_yticks([])
#ax.xaxis.set_ticks_position('none')
#ax.yaxis.set_ticks_position('none')

#fig.savefig("ch10-metro-graph.pdf")
#fig.savefig("ch10-metro-graph.png")
#fig.tight_layout()

![pic](pics/tokyo-metro.png)

In [95]:
# degree() = number of connections to a node for each node
#g.degree()

In [96]:
# find most highly connected station
d_max = max(g.degree().values())
d_max

6

In [97]:
[(n, d) for (n, d) in g.degree().items() if d == d_max]

[('M13', 6), ('G5', 6), ('Z4', 6), ('Y16', 6), ('N7', 6)]

In [98]:
# shortest path between Y24 & C19 stations
p = nx.shortest_path(g, "Y24", "C19")
np.array(p)

array(['Y24', 'Y23', 'Y22', 'Y21', 'Y20', 'Y19', 'Y18', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19'], 
      dtype='<U3')

In [99]:
# travel time from Y24 to C19 stations
np.sum([g[p[n]][p[n+1]]["weight"] 
        for n in range(len(p)-1) 
        if "weight" in g[p[n]][p[n+1]]])

35

In [100]:
# not 100% true - transfer nodes have no weights = zero transfer time (??)
# let's fix.
h = g.copy()

In [101]:
# assume 5 min transfer time to any transfer node
for n1, n2 in h.edges_iter():
    if "transfer" in h[n1][n2]:
        h[n1][n2]["weight"] = 5

In [102]:
p = nx.shortest_path(h, "Y24", "C19")
np.array(p)

array(['Y24', 'Y23', 'Y22', 'Y21', 'Y20', 'Y19', 'Y18', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19'], 
      dtype='<U3')

In [103]:
np.sum([h[p[n]][p[n+1]]["weight"] 
        for n in range(len(p)-1)])

85

In [104]:
# shortest path, Z1 -> H16
p = nx.shortest_path(h, "Z1", "H16")
np.array(p)

array(['Z1', 'Z2', 'Z3', 'Z4', 'Z5', 'Z6', 'Z7', 'Z8', 'Z9', 'G12', 'G13',
       'G14', 'G15', 'H16'], 
      dtype='<U3')

In [105]:
# travel time, Z1 -> H16
np.sum([h[p[n]][p[n+1]]["weight"] for n in range(len(p)-1)])

65

* NetworkX representation can be converted to adjacency matrix using to_scipy_sparse_matrix(), for analysis via **sp.csgraph**

In [106]:
# convert Tokyo Metro to adjacency matrix
A = nx.to_scipy_sparse_matrix(g)
A

<184x184 sparse matrix of type '<class 'numpy.int64'>'
	with 486 stored elements in Compressed Sparse Row format>

In [107]:
# compute adj.matrix's reverse Cuthill-McKee ordering
# (reduces max distance of matrix elements from diagonal, and
# permutes matrix with this ordering.)
perm = sp.csgraph.reverse_cuthill_mckee(A)

In [108]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
#ax1.spy(A, markersize=2)
#ax2.spy(sp_permute(A, perm, perm), markersize=2)
#fig.tight_layout()
#fig.savefig("ch12-rcm-graph.pdf")

![pic](pics/rcm-ordering.png)